# Benchmark Manifold GP Supervised Learning

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
import os
import scipy.spatial as ss
from time import time
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from gpytorch.priors import NormalPrior, GammaPrior

## Dataset Preprocessing

### Load & Settings

In [2]:
torch.manual_seed(1337)
dataset = 'mnist'

data = np.loadtxt('datasets/'+dataset+'_train.csv')
sampled_x, sampled_y = data[:, 2:], data[:, 1]
rand_idx = torch.randperm(sampled_x.shape[0])
sampled_x, sampled_y = sampled_x[rand_idx], sampled_y[rand_idx]
del rand_idx

data = np.loadtxt('datasets/'+dataset+'_test.csv')
test_x, test_y = data[:, 2:], data[:, 1]
    
preprocess = False
normalize_features = False
normalize_labels = True

In [3]:
if preprocess:
    # remove coincident points
    sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    sampled_y = sampled_y[id_unique]

    # cut between 0.1 and 0.9 percentile of distances
    import faiss
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(sampled_x)
    knn.add(sampled_x)
    v = np.sqrt(knn.search(sampled_x, 51)[0][:,1:])
    idx = np.argsort(v.mean(axis=1).ravel())
    percentile_start = int(np.round(idx.shape[0]*0.10))
    percentile_end = int(np.round(idx.shape[0]*0.90))
    sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
    sampled_y = sampled_y[idx[percentile_start:percentile_end]]
    del knn
m = sampled_x.shape[0]

### Trainset & Testset

In [4]:
split = int(0.1 * m)
train_x, train_y = sampled_x[:split], sampled_y[:split]

train_x, train_y = torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()
test_x, test_y = torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=3,
        nodes=train_x,
        neighbors=50,
        operator="randomwalk",
        modes=100,
        ball_scale=3.0,
        prior_bandwidth=False,
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel).to(device)

## Train

In [7]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.epsilon': 0.5,
    'covar_module.base_kernel.lengthscale': 0.5,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

In [8]:
t0 = time()
model.manifold_informed_train(lr=1e-2, iter=100, 
                              decay_step_size=1000, decay_magnitude=1.0, 
                              norm_step_size=10, norm_rand_vec=100, 
                              verbose=True, save=False)
t1 = time()
print("Time: %.2g sec" % (t1 - t0))

Iter: 0, LR: 0.100, Loss: 327.548, NoiseVar: 0.010, SignalVar: 49688.289, Lengthscale: 0.500, Epsilon: 0.500
Iter: 1, LR: 0.100, Loss: 337.466, NoiseVar: 0.009, SignalVar: 49688.188, Lengthscale: 0.462, Epsilon: 0.462
Iter: 2, LR: 0.100, Loss: 284.070, NoiseVar: 0.008, SignalVar: 49688.188, Lengthscale: 0.488, Epsilon: 0.427
Iter: 3, LR: 0.100, Loss: 273.730, NoiseVar: 0.008, SignalVar: 49688.148, Lengthscale: 0.507, Epsilon: 0.394
Iter: 4, LR: 0.100, Loss: 260.749, NoiseVar: 0.007, SignalVar: 49688.082, Lengthscale: 0.511, Epsilon: 0.363
Iter: 5, LR: 0.100, Loss: 210.878, NoiseVar: 0.006, SignalVar: 49688.004, Lengthscale: 0.503, Epsilon: 0.334
Iter: 6, LR: 0.100, Loss: 173.602, NoiseVar: 0.006, SignalVar: 49687.918, Lengthscale: 0.489, Epsilon: 0.307
Iter: 7, LR: 0.100, Loss: 143.143, NoiseVar: 0.005, SignalVar: 49687.828, Lengthscale: 0.473, Epsilon: 0.282
Iter: 8, LR: 0.100, Loss: 96.233, NoiseVar: 0.005, SignalVar: 49687.738, Lengthscale: 0.457, Epsilon: 0.258
Iter: 9, LR: 0.100, 

/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2.730250597000122 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iter: 13, LR: 0.100, Loss: -53.572, NoiseVar: 0.003, SignalVar: 49687.316, Lengthscale: 0.455, Epsilon: 0.165


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 4.644420146942139 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iter: 14, LR: 0.100, Loss: -68.785, NoiseVar: 0.003, SignalVar: 49687.234, Lengthscale: 0.462, Epsilon: 0.154


...

/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1.6392234563827515 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iter: 97, LR: 0.100, Loss: -50.724, NoiseVar: 0.000, SignalVar: 49679.012, Lengthscale: 0.462, Epsilon: 0.183


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1.5956169366836548 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iter: 98, LR: 0.100, Loss: -56.323, NoiseVar: 0.000, SignalVar: 49678.910, Lengthscale: 0.464, Epsilon: 0.182
Iter: 99, LR: 0.100, Loss: -22.734, NoiseVar: 0.000, SignalVar: 49678.809, Lengthscale: 0.466, Epsilon: 0.182
Time: 45 sec


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2.713918447494507 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


## Evaluation

In [9]:
%%capture
likelihood.eval()
model.eval()

## Metrics

In [10]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    preds_test = likelihood(model(test_x))
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    model._clear_cache()
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.5869, device='cuda:0')
NLL:  tensor(1617.9510, device='cuda:0')
